In [ ]:
%%script bash
# Install en_coref_lg if it hasn't been installed already.
pip show en_coref_lg 2>/dev/null;
if [ ! $? ]; then 
  pip install https://github.com/huggingface/neuralcoref-models/releases/download/en_coref_lg-3.0.0/en_coref_lg-3.0.0.tar.gz;
  pip show en_coref_lg 2>/dev/null;
fi

In [ ]:
# NLP
import spacy
from spacy import displacy
import en_coref_lg

# Data
import pandas as pd
import csv

# Model
import networkx as nx

# Utils
import time
from graphviz import Source
from tqdm import tqdm
import dill as pickle

In [ ]:
! ls ../data

In [ ]:
! head -n 2 ../data/document.csv

In [ ]:
documents = pd.read_csv('../data/document.csv')
documents.head()

In [ ]:
len(documents)

In [ ]:
nlp = en_coref_lg.load()

In [ ]:
%%time

parsed_documents = [nlp(row['document']) for _, row in tqdm(documents[0:100].iterrows(), total=100)]

In [ ]:
parsed_documents[0].__dir__()

In [ ]:
parsed_documents[0].print_tree()

In [ ]:
with open('documents.pkl', 'wb') as file:
    pickle.dump(parsed_documents, file)

In [ ]:
! ls -lah *.pkl

In [ ]:
for ent in parsed_documents[0].ents:
    print('{}: {}'.format(ent, ent.label_))